# blocs

In [1]:
import pandas as pd

In [2]:
BLOCS = [
    ['vote_type', 'mail'],
    ['vote_type', 'eday'],
    ['pgh_region', 'nside'],
    ['pgh_region', 'somon'],
    ['pgh_region', 'dt_oak_haz'],
    ['pgh_region', 'w14'],
    ['pgh_region', 'rem_ee'],
]

### 2021 Pittsburgh mayoral primary

In [3]:
pghmayoral21 = pd.DataFrame(BLOCS, columns=['class', 'type'])

pghmayoral21['pct_gain21'] = ''
pghmayoral21['pct_pedu21'] = ''
pghmayoral21['pct_othr21'] = ''

### Results

In [4]:
dfblocs = pd.DataFrame(BLOCS, columns=['class', 'type'])
pd.merge(dfblocs, pghmayoral21, on=['class', 'type'])

,class,type,pct_gain21,pct_pedu21,pct_othr21
0,vote_type,mail,,,
1,vote_type,eday,,,
2,pgh_region,nside,,,
3,pgh_region,somon,,,
4,pgh_region,dt_oak_haz,,,
5,pgh_region,w14,,,
6,pgh_region,rem_ee,,,
